In [1]:
import pygame
import random
import sys
import unittest

pygame.init()

# Constants
CARD_WIDTH = 60
CARD_HEIGHT = 80
MARGIN = 5  
COLUMNS = 10
ROWS = 5
MARGIN_TOP = 100  # Space at the top for the tracker
MARGIN_LEFT = MARGIN
SCREEN_WIDTH = COLUMNS * (CARD_WIDTH + MARGIN) + MARGIN_LEFT + MARGIN
SCREEN_HEIGHT = MARGIN_TOP + ROWS * (CARD_HEIGHT + MARGIN) + MARGIN

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption('Memory Matching Game')

# Colors
BACKGROUND_COLOR = (30, 30, 30)
CARD_BACK_COLOR = (50, 50, 200)
CARD_FRONT_COLOR = (200, 200, 200)
TEXT_COLOR = (255, 255, 255)

# Create a list of pairs of symbols
symbols = ['&', '$', '?', '!', '€', '»', '~', '☼', '☺', '☻', '#', '^', '{', '[', ';', '@', '+', '=', '|', '<', '♠', '♣', '♥', '♦', '%']
card_values = symbols * 2  
random.shuffle(card_values)

# Create the cards
cards = []
for row in range(ROWS):
    for col in range(COLUMNS):
        x = MARGIN_LEFT + col * (CARD_WIDTH + MARGIN)
        y = MARGIN_TOP + row * (CARD_HEIGHT + MARGIN)
        rect = pygame.Rect(x, y, CARD_WIDTH, CARD_HEIGHT)
        value = card_values.pop()
        card = {'rect': rect, 'value': value, 'matched': False, 'face_up': False}
        cards.append(card)

# Initialize variables
first_card = None
second_card = None
waiting = False
wait_time = 1000  
wait_start_time = None
game_over = False
attempts = 0
matches = 0

# Timer variables
timer_duration = 200 * 1000  # 200 seconds in milliseconds
timer_start_time = None
timer_running = False

clock = pygame.time.Clock()

# Use a font that supports Unicode symbols
font_name = pygame.font.match_font('segoeuisymbol, Arial Unicode MS, Arial')
font = pygame.font.Font(font_name, 36)

running = True
while running:
    clock.tick(30)  

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN and not waiting and not game_over:
            pos = pygame.mouse.get_pos()
            for card in cards:
                if card['rect'].collidepoint(pos) and not card['face_up'] and not card['matched']:
                    card['face_up'] = True
                    if not first_card:
                        first_card = card
                        if not timer_running:  # Start the timer when the first card is flipped
                            timer_start_time = pygame.time.get_ticks()
                            timer_running = True
                    elif not second_card:
                        second_card = card
                        waiting = True
                        wait_start_time = pygame.time.get_ticks()
                        attempts += 1  
                    break  

    # Timer logic
    if timer_running:
        elapsed_time = pygame.time.get_ticks() - timer_start_time
        remaining_time = max(0, timer_duration - elapsed_time)
        if remaining_time == 0:
            game_over = True

    # Check if we are waiting to flip back cards
    if waiting:
        current_time = pygame.time.get_ticks()
        if current_time - wait_start_time >= wait_time:
            # Check if the two cards match
            if first_card['value'] == second_card['value']:
                first_card['matched'] = True
                second_card['matched'] = True
                matches += 1  
            else:
                first_card['face_up'] = False
                second_card['face_up'] = False
            # Reset the selected cards
            first_card = None
            second_card = None
            waiting = False

    # Check for game completion
    if not game_over and all(card['matched'] for card in cards):
        game_over = True
        timer_running = False  # Stop the timer

    # Draw the background
    screen.fill(BACKGROUND_COLOR)

    # Draw the timer
    if timer_running:
        timer_text = font.render(f'Time Left: {remaining_time // 1000}', True, TEXT_COLOR)
        timer_rect = timer_text.get_rect(topleft=(MARGIN, MARGIN))
        screen.blit(timer_text, timer_rect)

    # Draw the attempts and matches tracker
    attempts_text = font.render(f'Attempts: {attempts}', True, TEXT_COLOR)
    matches_text = font.render(f'Matches: {matches}', True, TEXT_COLOR)
    attempts_rect = attempts_text.get_rect(topright=(SCREEN_WIDTH - MARGIN, MARGIN))
    matches_rect = matches_text.get_rect(topright=(SCREEN_WIDTH - MARGIN, attempts_rect.bottom + 5))
    screen.blit(attempts_text, attempts_rect)
    screen.blit(matches_text, matches_rect)

    # Draw the cards
    for card in cards:
        if card['face_up'] or card['matched']:
            pygame.draw.rect(screen, CARD_FRONT_COLOR, card['rect'])
            # Draw the symbol
            text = font.render(str(card['value']), True, (0, 0, 0))
            text_rect = text.get_rect(center=card['rect'].center)
            screen.blit(text, text_rect)
        else:
            pygame.draw.rect(screen, CARD_BACK_COLOR, card['rect'])

    # If game over, draw game over or win message
    if game_over:
        font_large = pygame.font.Font(font_name, 72)
        if matches == len(cards) // 2:  # All matches found
            text = font_large.render("You Win!", True, (255, 255, 255)) #Beat the Timer
        else:
            text = font_large.render("Game Over!", True, (255, 255, 255)) # Did not beat the timer
        text_rect = text.get_rect(center=(SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2))
        screen.blit(text, text_rect)

    # Update the display
    pygame.display.flip()

pygame.quit()
sys.exit()

# Test Suite
class TestMemoryGame(unittest.TestCase):

    def setUp(self):
        self.cards = create_cards()
        self.first_card = self.cards[0]
        self.second_card = self.cards[1]
        self.second_card['value'] = self.first_card['value']  

    def test_create_cards(self):
        self.assertEqual(len(self.cards), COLUMNS * ROWS)
        values = [card['value'] for card in self.cards]
        self.assertEqual(len(values), len(set(values)))  

    def test_check_for_match(self):
        self.assertTrue(check_for_match(self.first_card, self.second_card))
        self.second_card['value'] = 'X'  
        self.assertFalse(check_for_match(self.first_card, self.second_card))

    def test_game_completed(self):
        for card in self.cards:
            card['matched'] = True
        self.assertTrue(game_completed(self.cards))
        
        self.cards[0]['matched'] = False  
        self.assertFalse(game_completed(self.cards))

# Allow Jupyter Notebook
unittest.main(argv=[''], exit=False)


pygame 2.6.0 (SDL 2.28.4, Python 3.12.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

/Users/colleenfisher/Library/Python/3.12/lib/python/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
